# **<mark>Deadlock</mark>**

Bế tắc (Deadlock) là một vấn đề đồng thời trong đó hai phiên chặn tiến trình của nhau. Phiên đầu tiên có khóa tài nguyên mà phiên kia muốn truy cập và ngược lại.

![Deadlock](.\SQL-Server-Deadlock.png)

Trong hình này, các invoices và invoice\_items là các bảng.

- Đầu tiên, phiên một truy cập vào bảng invoices và khóa nó.
- Thứ hai, phiên hai khóa bảng invoice\_items và khóa nó.
- Thứ ba, phiên một muốn truy cập vào bảng invoice\_items nhưng cần đợi phiên hai hoàn tất. Đồng thời, phiên hai muốn truy cập bảng invoices nhưng cần đợi phiên hai hoàn thành.

Do đó, hai phiên đang chờ nhau cho đến khi SQL Server chủ động kết thúc một trong số chúng. Phiên bị kết thúc bởi SQL Server được gọi là nạn nhân của bế tắc (**deadlock victim**).

In [1]:
CREATE TABLE invoices (
  id int IDENTITY PRIMARY KEY,
  customer_id int NOT NULL,
  total decimal(10, 2) NOT NULL DEFAULT 0 CHECK (total >= 0)
)

CREATE TABLE invoice_items (
  id int,
  invoice_id int NOT NULL,
  item_name varchar(100) NOT NULL,
  amount decimal(10, 2) NOT NULL CHECK (amount >= 0),
  tax decimal(4, 2) NOT NULL CHECK (tax >= 0),
  PRIMARY KEY (id, invoice_id),
  FOREIGN KEY (invoice_id) REFERENCES invoices (id)
     ON UPDATE CASCADE
     ON DELETE CASCADE
)

INSERT INTO invoices (customer_id, total)
  VALUES (100, 0)

INSERT INTO invoice_items (id, invoice_id, item_name, amount, tax)
  VALUES (10, 1, 'Keyboard', 70, 0.08),
  (20, 1, 'Mouse', 50, 0.08)

UPDATE invoices
SET total = (SELECT
  SUM(amount * (1 + tax))
  FROM invoice_items
  WHERE invoice_id = 1
)

(1 row affected)

(2 rows affected)

(1 row affected)

Total execution time: 00:00:00.011

Chạy lần lượt 2 session theo bảng sau:

![Deadlock-Demo](Deadlock_Sessions.png)

In [ ]:
-- Session 1
BEGIN TRAN

UPDATE invoices
SET customer_id = 100
WHERE id = 1

UPDATE invoice_items
SET item_name = 'Cool Keyboard'
WHERE id = 10

In [ ]:
-- Session 2
BEGIN TRAN

UPDATE invoice_items
SET amount = 100
WHERE id = 10

UPDATE invoices
SET total = (SELECT
SUM(amount * (1 + tax))
FROM invoice_items
WHERE invoice_id = 1)
WHERE id = 1